<a href="https://colab.research.google.com/github/seinJOO/PythonClass/blob/main/Python_Basic/0531/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

### TensorBoard로 시각화하기

# 데이터 플로우 그래프 구축하기
a = tf.constant(20, name="a")
b = tf.constant(30, name="b")
mul_op = a*b

# 세션 생성
sess = tf.Session()

# TensorBoard 사용
tw = tf.summary.FileWriter("log_dir", graph=sess.graph)

# 세션 실행
print(sess.run(mul_op))

Instructions for updating:
non-resource variables are not supported in the long term
600


In [ ]:


# 상수와 변수 선언
a = tf.constant(100, name="a")
b = tf.constant(200, name="b")
c = tf.constant(300, name="c")
v = tf.Variable(0, name="v")

# 곱셈을 수행하는 그래프 정의
calc_op = a + b * c
assign_op = tf.assign(v, calc_op)

# 세션 생성
sess = tf.Session()

# TensorBoard 사용
tw = tf.summary.FileWriter("log_dir", graph = sess.graph)

# 세션 실행하기 (5)
sess.run(assign_op)
print(sess.run(v))

60100


In [ ]:
# TensorFlow를 활용한 학습

import pandas as pd
import numpy as np

csv = pd.read_csv('/content/sample_data/bmi.csv')

# 데이터 정규화
csv['height'] = csv['height']/200
csv['weight'] = csv['weight']/100

## 레이블을 배열로 변환해보기
# thin : [1,0,0] / normal : [0,1,0] / fat : [0,0,1]
bclass = {'thin' : [1,0,0], 'normal' : [0,1,0], 'fat' : [0,0,1]}
csv['label_pat'] = csv['label'].apply(lambda x : np.array(bclass[x]))

# 테스트를 위한 데이터 분류
test_csv = csv[15000:20000]
test_pat = test_csv[['weight', 'height']]   # 키랑 몸무게만 추출
test_ans = list(test_csv['label_pat'])      # 배열로 변환한 레이블 저장

# 데이터 플로우 그래프 구축하기(x축, y축)
# 플레이스홀더 선언
x = tf.placeholder(tf.float32, [None, 2], name="x")   # 키와 몸무게 데이터 - 가변행 * 2열 행렬
y_ = tf.placeholder(tf.float32, [None, 3], name="y_")   # 정답 레이블 넣기 - 가변행 * 3열 행렬

# 변수 선언
with tf.name_scope('interface') as scope :
  w = tf.Variable(tf.zeros([2,3]), name="w")    # 가중치 부여 - 2*3 행렬
  b = tf.Variable(tf.zeros([3]), name="b")      # 바이어스

  # 소프트맥스 회귀 정의
  with tf.name_scope('softmax') as scope :
    y = tf.nn.softmax(tf.matmul(x,w) + b)       # matmul : matrix multiple (행렬곱)

# 모델 훈련
with tf.name_scope('loss') as scope : 
  cross_entropy = -tf.reduce_sum(y_ * tf.log(y))      # y의 교차엔트로피(loss계산)
with tf.name_scope('training') as scope :
  optimizer = tf.train.GradientDescentOptimizer(0.01) # 학습계수 0.01 : 오차 함수가 최대가 되도록 함(경사하강법)
  train = optimizer.minimize(cross_entropy)
      # optimizer, train을 이렇게 정의하면 TensorFlow가 가중치 w와 바이어스 b값을 자동으로 변경해줌

# 정답률 구하기
with tf.name_scope('accuracy') as scope :
  predict = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

# 세션 시작
with tf.Session() as sess :
  tw = tf.summary.FileWriter("log_dir", graph = sess.graph)
  sess.run(tf.global_variables_initializer())     # 변수 초기화

  # 학습시키기 15000
  for step in range(3500) :
      i = (step * 100) % 14000            # 15000번째 행부터는 테스트용 데이터로 빼둠
      rows = csv[1 + i : 1 + i + 100]     # 100개씩
      x_pat = rows[['weight','height']]   # 데이터 입력
      y_ans = list(rows['label_pat'])     # label 입력
      fd = {x : x_pat, y_ : y_ans}
      sess.run(train, feed_dict = fd)
      if step % 500 == 0 :
          cre = sess.run(cross_entropy, feed_dict = fd)
          acc = sess.run(accuracy, feed_dict = {x : test_pat, y_ : test_ans})
          print('step =', step, 'cre =', cre, 'acc =', acc)
          
  # 최종 정답률
  acc = sess.run(accuracy, feed_dict = { x : test_pat, y_ : test_ans })
  print('정답률 =', acc)

step = 0 cre = 108.59491 acc = 0.3734
step = 500 cre = 53.11373 acc = 0.8638
step = 1000 cre = 45.682495 acc = 0.8888
step = 1500 cre = 40.71729 acc = 0.9304
step = 2000 cre = 35.59798 acc = 0.9554
step = 2500 cre = 28.15816 acc = 0.9612
step = 3000 cre = 28.311062 acc = 0.9496
정답률 = 0.964


In [ ]:
rm log_dir/*

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./log_dir/

In [ ]:
### TensorBoard로 딥러닝
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
tf.disable_v2_behavior()

mnist = tf.keras.datasets.mnist
pixels = 28 * 28
nums = 10   # 0-9 사이의 카테고리
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 플레이스홀더 정의
x = tf.placeholder(tf.float32, shape=[None, pixels], name="x") # 이미지 데이터
y_ = tf.placeholder(tf.float32, shape=[None, nums], name="y_") # 정답 레이블

# 가중치와 바이어스를 초기화하는 함수
def weight_variable(name, shape):
  W_init = tf.truncated_normal(shape, stddev=0.1)
  W = tf.Variable(W_init, name="W_"+name)
  return W



10000